In [ ]:
%pylab

In [ ]:
%matplotlib inline

In [ ]:
import factory, simulate, scipy, re, os, GPy
import pandas as pd

In [ ]:
# pattern = 'simulation-([0-9]+)-([0-9]+)-([0-9]+)-([0-9.]+)-([0-9.]+)-([0-9.]+)-([a-zA-Z.0-9]+).csv'
pattern = 'simulation-([0-9.]+-)*([a-zA-Z.0-9]+).csv'

targets = os.listdir("results/")
targets = filter(lambda x: re.match(pattern,x), targets)
targets = map(lambda x: re.match(pattern, x).groups()[-1], targets)

targets = list(set(targets))
targets.sort()
targets

In [ ]:
pattern = 'simulation-([0-9]+)-([0-9]+)-([0-9]+)-([0-9.]+)-([0-9.]+)-([0-9.]+)-%s.csv'

for target in targets:

    files = os.listdir("results/")
    files = filter(lambda x: re.match(pattern%target,x), files)
    
    print target, files
    print

In [ ]:
def findFiles(target, pattern='simulation-([0-9]+)-([0-9]+)-([0-9]+)-([0-9.]+)-([0-9.]+)-([0-9.]+)-%s.csv'):
    files = os.listdir("results/")
    files = filter(lambda x: re.match(pattern%target,x), files)
    return files

In [ ]:
def loadResults(target, files=None, pattern='simulation-([0-9]+)-([0-9]+)-([0-9]+)-([0-9.]+)-([0-9.]+)-([0-9.]+)-%s.csv'):
    
    if files is None:
        files = findFiles(target, pattern)
    
    results = {}
    for f in files:
        nobs, nbatch, nrep, sigma, batchVariance, repVariance = re.match(pattern%target,f).groups()
        results[(nobs, nbatch, nrep, sigma, batchVariance, repVariance)] = pd.read_csv('results/%s'%f, header=[0,1], index_col=None)
        
    return results

In [ ]:
def plotResult(res, target, title=False, prefix=None):
    
    if type(res) == dict:
        for k,v in res.iteritems():
            plt.figure(figsize=(10,4))
            plotResult(v.values, target)
            if title:
                plt.suptitle(k,y=1.05)
                
            if not prefix is None:
                plt.savefig("%s-%s-%s.pdf"%(prefix, str(k),target), bbox_inches='tight')
        
        return

    for i in range(4):

        plt.subplot(1,4,i+1)
        plt.title("M%d"%i)

        plt.boxplot(res[:,i*4:(i+1)*4],
                       labels=['M%d'%i for i in  range(4)]);

        if 'accuracy' in target:
            plt.plot([1,4], [.95,.95])

        plt.ylim(np.nanmin(res), np.nanmax(res))

    plt.tight_layout()

In [ ]:
def plotBayesFactor(res, title=False):
    
    if type(res) == dict:
        for k,v in res.iteritems():
            plt.figure(figsize=(10,4))
            plotBayesFactor(v.values)
            if title:
                plt.suptitle(k)
        
        return
    
    ylim = (np.inf, -np.inf)

    for i in range(4):

        plt.subplot(1,4,i+1)
        plt.title("M%d"%i)
        
        plt.boxplot(res[:,i*4:(i+1)*4] - res[:,i*4][:,None],
                       labels=['M%d'%i for i in  range(4)]);

#         plt.boxplot(res[:,i*4:(i+1)*4],
#                        labels=['M%d'%i for i in  range(4)]);

        if 'accuracy' in target:
            plt.plot([1,4], [.95,.95])
            
        ylim = (min(ylim[0], np.nanmin(res[:,i*4:(i+1)*4] - res[:,i*4][:,None])), max(ylim[1], np.nanmax(res[:,i*4:(i+1)*4] - res[:,i*4][:,None])))

    for i in range(4):

        plt.subplot(1,4,i+1)
        plt.ylim(ylim)

    plt.tight_layout()

In [ ]:
for t in targets:
    res = loadResults(t)
    plotResult(res,t,title=True,prefix='figures/simulation')

In [ ]:
c = res[res.keys()[0]].columns

In [ ]:
c.names = ['source', 'model']
c

In [ ]:
t = res[res.keys()[0]]
t.columns.names = ['source', 'model']
t